In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/diabetes-health-indicators-dataset/diabetes_binary_5050split_health_indicators_BRFSS2015.csv
/kaggle/input/diabetes-health-indicators-dataset/diabetes_binary_health_indicators_BRFSS2015.csv
/kaggle/input/diabetes-health-indicators-dataset/diabetes_012_health_indicators_BRFSS2015.csv


In [2]:
!pip install MiniSom

  Preparing metadata (setup.py) ... done
  Created wheel for MiniSom: filename=MiniSom-2.3.5-py3-none-any.whl size=12031 sha256=d27eccadc6055fc7e0619697e7d9fa2cb3afad53168dac4eface0def8f70f6c7
  Stored in directory: /root/.cache/pip/wheels/19/db/95/5e53bc2b88a328217fdf9f2886cafbe86b0df274f4b601f572
Successfully built MiniSom


In [3]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from minisom import MiniSom
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from minisom import MiniSom
import random

In [4]:
df = pd.read_csv("/kaggle/input/diabetes-health-indicators-dataset/diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
df.head()


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       70692 non-null  float64
 1   HighBP                70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   Stroke                70692 non-null  float64
 7   HeartDiseaseorAttack  70692 non-null  float64
 8   PhysActivity          70692 non-null  float64
 9   Fruits                70692 non-null  float64
 10  Veggies               70692 non-null  float64
 11  HvyAlcoholConsump     70692 non-null  float64
 12  AnyHealthcare         70692 non-null  float64
 13  NoDocbcCost           70692 non-null  float64
 14  GenHlth               70692 non-null  float64
 15  MentHlth           

In [6]:
df.describe()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
count,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,...,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000,70692.000000
mean,0.500000,0.563458,0.525703,0.975259,29.856985,0.475273,0.062171,0.147810,0.703036,0.611795,...,0.954960,0.093914,2.837082,3.752037,5.810417,0.252730,0.456997,8.584055,4.920953,5.698311
std,0.500004,0.495960,0.499342,0.155336,7.113954,0.499392,0.241468,0.354914,0.456924,0.487345,...,0.207394,0.291712,1.113565,8.155627,10.062261,0.434581,0.498151,2.852153,1.029081,2.175196
min,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,1.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,7.000000,4.000000,4.000000
50%,0.500000,1.000000,1.000000,1.000000,29.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,9.000000,5.000000,6.000000
75%,1.000000,1.000000,1.000000,1.000000,33.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,4.000000,2.000000,6.000000,1.000000,1.000000,11.000000,6.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,5.000000,30.000000,30.000000,1.000000,1.000000,13.000000,6.000000,8.000000


In [7]:
df['Diabetes_binary'].value_counts()

Diabetes_binary
0.0    35346
1.0    35346
Name: count, dtype: int64

In [8]:
model = LogisticRegression(max_iter = 1000)
X = df.drop(columns = 'Diabetes_binary')
y = df['Diabetes_binary']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify = y,random_state=42)
model.fit(X_train,y_train)
preds = model.predict(X_test)
accuracy_score(preds,y_test)

0.7458094631869298

In [9]:
columns = X.columns.tolist()
columns

['HighBP',
 'HighChol',
 'CholCheck',
 'BMI',
 'Smoker',
 'Stroke',
 'HeartDiseaseorAttack',
 'PhysActivity',
 'Fruits',
 'Veggies',
 'HvyAlcoholConsump',
 'AnyHealthcare',
 'NoDocbcCost',
 'GenHlth',
 'MentHlth',
 'PhysHlth',
 'DiffWalk',
 'Sex',
 'Age',
 'Education',
 'Income']

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)
 
NUM_FEATURES = X_train.shape[1]
POP_SIZE = 20
N_GENERATIONS = 20
MUTATION_PROB = 0.2
NUM_ISLANDS = 3
MIGRATION_INTERVAL = 5
MIGRATION_SIZE = 2
ISLAND_POP_SIZE = POP_SIZE // NUM_ISLANDS

def init_population(size):
    return [np.random.randint(0, 2, NUM_FEATURES) for _ in range(size)]

def evaluate(individual):
    selected = np.where(individual == 1)[0]
    if len(selected) == 0:
        return 0
    clf = LogisticRegression(max_iter=1000, solver='liblinear')
    clf.fit(X_train[:, selected], y_train)
    preds = clf.predict(X_test[:, selected])
    return accuracy_score(y_test, preds)

def tournament_selection(pop, scores, k=3):
    selected = np.random.choice(len(pop), k, replace=False)
    best = max(selected, key=lambda i: scores[i])
    return pop[best]

def crossover(parent1, parent2):
    point = np.random.randint(1, NUM_FEATURES - 1)
    return np.concatenate((parent1[:point], parent2[point:]))

def mutate(individual):
    if np.random.rand() < MUTATION_PROB:
        idx = np.random.randint(0, NUM_FEATURES)
        individual[idx] = 1 - individual[idx]
    return individual

islands = [init_population(ISLAND_POP_SIZE) for _ in range(NUM_ISLANDS)]

global_best_individual = None
global_best_score = 0

for gen in range(N_GENERATIONS):
    for i in range(NUM_ISLANDS):
        pop = islands[i]
        scores = [evaluate(ind) for ind in pop]
        new_pop = []

   
        best_idx = np.argmax(scores)
        new_pop.append(pop[best_idx].copy())

        while len(new_pop) < ISLAND_POP_SIZE:
            p1 = tournament_selection(pop, scores)
            p2 = tournament_selection(pop, scores)
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)

        islands[i] = new_pop

      
        best_score = scores[best_idx]
        if best_score > global_best_score:
            global_best_score = best_score
            global_best_individual = pop[best_idx].copy()


    if (gen + 1) % MIGRATION_INTERVAL == 0:
        for i in range(NUM_ISLANDS):
            donor_idx = np.random.choice(ISLAND_POP_SIZE, MIGRATION_SIZE, replace=False)
            donor = [islands[i][j] for j in donor_idx]
            target_island = islands[(i + 1) % NUM_ISLANDS]
            target_island[:MIGRATION_SIZE] = donor


selected_features = np.where(global_best_individual == 1)[0]
print("Best Accuracy:", global_best_score)
print("Selected Feature Indices:", selected_features)


Best Accuracy: 0.7434047669566447
Selected Feature Indices: [ 0  2  3  5  9 10 12 13 14 15 17 18 20]


In [11]:

n_particles = 20
n_iterations = 30
rank = 5  
lambda_classification = 0.5  

def initialize_particles(n_particles, m, r, n):
    # Flatten W (m×r) and H (r×n)
    dim = m * r + r * n
    particles = np.random.rand(n_particles, dim)
    velocities = np.zeros_like(particles)
    return particles, velocities

def evaluate_particle(particle, m, r, n, X_train, y_train, X_test, y_test):
    W_flat = particle[:m * r]
    H_flat = particle[m * r:]
    W = W_flat.reshape(m, r)
    H = H_flat.reshape(r, n)

    X_reconstructed = np.dot(W, H)
    frob_loss = np.linalg.norm(X_train - X_reconstructed, ord='fro') ** 2

    X_proj = W
    clf = LogisticRegression(max_iter=1000, solver='liblinear')
    try:
        clf.fit(X_proj, y_train)
        y_pred = clf.predict(W)
        class_loss = 1 - accuracy_score(y_train, y_pred)
    except:
        class_loss = 1.0  # worst case if model fails

    return frob_loss + lambda_classification * class_loss


m, n = X_train.shape
particles, velocities = initialize_particles(n_particles, m, rank, n)


pbest = particles.copy()
pbest_scores = np.array([evaluate_particle(p, m, rank, n, X_train, y_train, X_test, y_test) for p in pbest])
gbest_idx = np.argmin(pbest_scores)
gbest = pbest[gbest_idx].copy()

w = 0.5     
c1 = 1.5   
c2 = 1.5    

print("Running PSO...")

for it in range(n_iterations):
    for i in range(n_particles):
      
        r1 = np.random.rand(particles.shape[1])
        r2 = np.random.rand(particles.shape[1])
        velocities[i] = (w * velocities[i] +
                         c1 * r1 * (pbest[i] - particles[i]) +
                         c2 * r2 * (gbest - particles[i]))

     
        particles[i] += velocities[i]
        particles[i] = np.clip(particles[i], 0, 1)

  
        fitness = evaluate_particle(particles[i], m, rank, n, X_train, y_train, X_test, y_test)

    
        if fitness < pbest_scores[i]:
            pbest[i] = particles[i].copy()
            pbest_scores[i] = fitness

    gbest_idx = np.argmin(pbest_scores)
    gbest = pbest[gbest_idx].copy()

    print(f"Iteration {it+1}/{n_iterations}, Best Fitness: {pbest_scores[gbest_idx]:.4f}")

W_best = gbest[:m * rank].reshape(m, rank)
H_best = gbest[m * rank:].reshape(rank, n)
X_proj = W_best

clf_final = LogisticRegression(max_iter=1000, solver='liblinear')
clf_final.fit(X_proj, y_train)
final_preds = clf_final.predict(X_proj)
final_acc = accuracy_score(y_train, final_preds)

print(f"\nFinal Classification Accuracy on Train (projected): {final_acc:.4f}")


Running PSO...
Iteration 1/30, Best Fitness: 2638057.9544
Iteration 2/30, Best Fitness: 2527428.5139
Iteration 3/30, Best Fitness: 2441543.0073
Iteration 4/30, Best Fitness: 2369842.7730
Iteration 5/30, Best Fitness: 2298835.7593
Iteration 6/30, Best Fitness: 2181219.7033
Iteration 7/30, Best Fitness: 2172527.7756
Iteration 8/30, Best Fitness: 2125935.8230
Iteration 9/30, Best Fitness: 2082187.7382
Iteration 10/30, Best Fitness: 2052572.9003
Iteration 11/30, Best Fitness: 2012146.9993
Iteration 12/30, Best Fitness: 1995717.6469
Iteration 13/30, Best Fitness: 1957632.6467
Iteration 14/30, Best Fitness: 1939072.0247
Iteration 15/30, Best Fitness: 1911089.6262
Iteration 16/30, Best Fitness: 1894160.7869
Iteration 17/30, Best Fitness: 1878612.5786
Iteration 18/30, Best Fitness: 1866204.7650
Iteration 19/30, Best Fitness: 1859668.9323
Iteration 20/30, Best Fitness: 1851719.3219
Iteration 21/30, Best Fitness: 1847003.9859
Iteration 22/30, Best Fitness: 1840819.8072
Iteration 23/30, Best Fitn